In [124]:
# Imports
import numpy as np
import pandas as pd
import pyspark
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import contractions
import math
from itertools import chain
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
from nltk.stem import WordNetLemmatizer

import re
import string
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

from google.cloud import bigquery

from sklearn.feature_extraction.text import TfidfVectorizer 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jupyter/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [125]:
df = pd.read_csv("flexfits_emails_2022.csv")

In [126]:
df_1= pd.read_csv("flexfits_emails_2022.csv")

In [127]:
GROUP_CUSTOM_STOPWORDS = [
    "bike", "nbsp", "crn","sample", "bikes", "great", "good", "love","like", "recommend","smells","collected","part","review","promotion","razor",
    "shave","shaving","blade","camille","cantu","daughter","today","kccc","jessie","miss","fabric","pillow","softener","or",    "tiktok","OR",
     'Proov', 'proov','Natalist', 'natalist','Stix', 'stix','Clear Blue', 'blue','Modern Fertility', 'modern', 'free', 'app', 'mf', 'android', 'pinchme',
    'Pregmate', 'pregmate', 'received','First Response', 'test', 'ovulation', 'none', 'see', 'store', 'day', 'month', 'amy', 'face', 'far', 'scan',
    'it‚äôs','don‚äôt', 'using_minoxidil', '_oz','i‚äôm','moreread','using','stating',
    'doesn‚äôt', 'classic','order','ordering', 'best', 'hard', 'seltzer', 'claw','never','got','ordered','white','product','high-noon','bottle','girl',
    'read_lessread','read_moreread','lessread_le','le_stating','moreread_stating','scalp detox','le_serum',
    'moreread_serum','i’ve_stem','hair_read','don’t_know','i’ve_bought', 'liquid_kinda', "i'ḿ"'use_serum','le_mus','read_serum','ingredient_us','oz_extreme',
    'cell_serum','cell','stem','le','read','lessread','moreread','more','multi-colored—from_brown','findin_read','strengthening_hair','hair_oil','care_oil',
    'hair','oil_hair','extreme_hair','hair_care','extreme','oz','_strengthening','used twice', 'oil_oil', 'customer',    'nan', 'read_moreread', 'noreferrer',
    'noreferrer_farmer','farmer_dog/a','noreferrer_ollie/a','rel','noopener','blank_rel',
    'rel_nofollow','blank','nofollow','target_blank','noopener_noreferrer','nofollower_noopener','nofollow_noopener',"partake","partake_cooky", "crunchy_cooky","cooky_vegan","madegood","apple_jack"
]

added_stopwords_li = [
    "it’s",    "'d",    "'s",
    "n't",    "'m",
    "i've",    "it's",    "'ve",
    "'re",    "'ll",    "``",    "''",    "...",    "--", "https",
    "voxbox",    "influenster",    "cracker",    "or",    'it‚äôs',
    'don‚äôt',    'i‚äôm',    'doesn‚äôt',    "get", "also",    "even",    "since", "amy"
]

REV_COLS = ['source_name',
    'normalized_url',
    'review_source_id',
    'reviewer_source_id',
    'brand_name',
    'product_name',
    'product_source_id',
    'review_date',
    'review_rating',
    'review_content'
]

product_categories = ['Accessories',
'Alcoholic Beverages',
'Apparel & Footwear',
'Food',
'Food Establishments',
'Health & Beauty Establishments',
'Household Consumables',
'Household Durables',
'Other',
'Pet Products',
'Non-Alcoholic Beverages',
'Personal Care & Beauty',
'Retailers',
'Sports & Outdoors'
]

drop_words = ["kinship",
              "mineral",
              "trendmood",
              "hue_stick",
              "huesticks",
              "stick",
              "blume",
              "daydreamer",
              "loli",
              "plum",
              "elixir",
              "nut",
              "beekman",
              "goat",
              "milk",
              "bloom",
              "fifth_root",
              "fifth",
              "lip",
              "cannuka",
              "review",
              "zitsticka",
              "discipline",
              "uoma",
              "bv",
              "yeast",
              "bye_bloat",
              "tula",
              "tula_product",
              "huestick",
              "hue",
              "live_tinted",
              "meltdown",
              "loli_product",
              "bloom_cream",
              "beekman_product",
              "pure_goat",
              "karmic_cleanse",
              
             ]

MIN_TERM_PCT = 0

MAX_TERM_PCT = 100

TERM_PERCENT_HEAD = 1

TOP_N_WORDS = 8

np.random.seed(42) # set seed for models for reproducibility

stopwords_li = stopwords.words('english')
punkts_li = list(string.punctuation)



In [128]:
def tokenize(s, strip_str="=-_/\+.:,'* 1234567890—"):
    custom_words_li = GROUP_CUSTOM_STOPWORDS + added_stopwords_li+stopwords_li + punkts_li + [""]
    return [w.lower().strip(strip_str) for w in word_tokenize(str(s)) if w.lower().strip(strip_str) not in custom_words_li]

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in tokens]

def unique_list(li):
    deduped_li = list(set(li))
    return deduped_li

def bigram(tokens):
    bi_tup = list(bigrams(tokens))
    bi_li = ['_'.join(tup) for tup in bi_tup]
    return bi_li

In [180]:
#remove repeat emails

df = df["Description"].drop_duplicates()
df = df.reset_index()

In [181]:
df

,index,Description
0,0,"\n?\nHi,\nI was hoping to speak with you about..."
1,1,\n\n------------------\nSubmitted from: https...
2,2,\n\n------------------\nSubmitted from: https...
3,3,"\n""Mothering our Mothers"" workshop Spring 2019..."
4,4,"( I wrote you before, but failed to notice I ..."
...,...,...
3734,3734,Can I please cancel monthly shipping order? I ...
3735,3735,Can I please cancel my account? I am no longer...
3736,3736,Can I please cancel my account? I no longer ne...
3737,3737,Can I please cancel my account. \nThank you!\n...


In [182]:
df["tokenized_email"] = df["Description"].apply(lambda x: lemmatize(tokenize(x)))

In [183]:
df

,index,Description,tokenized_email
0,0,"\n?\nHi,\nI was hoping to speak with you about...","[hi, hoping, speak, ultra-luxury, community, f..."
1,1,\n\n------------------\nSubmitted from: https...,"[submitted, flexfits.com/account]"
2,2,\n\n------------------\nSubmitted from: https...,"[submitted, flexfits.com/blogs/thefixx/10-tips..."
3,3,"\n""Mothering our Mothers"" workshop Spring 2019...","[mothering, mother, workshop, spring, atlanta,..."
4,4,"( I wrote you before, but failed to notice I ...","[wrote, failed, notice, wrote, wrong, email, w..."
...,...,...,...
3734,3734,Can I please cancel monthly shipping order? I ...,"[please, cancel, monthly, shipping, many, righ..."
3735,3735,Can I please cancel my account? I am no longer...,"[please, cancel, account, longer, need, thank,..."
3736,3736,Can I please cancel my account? I no longer ne...,"[please, cancel, account, longer, need, flex, ..."
3737,3737,Can I please cancel my account. \nThank you!\n...,"[please, cancel, account, thank, beleive, paid..."


In [184]:
df = df.dropna(subset=['tokenized_email'])

In [185]:
df['unibi'] = df['tokenized_email'].apply(lambda x: unique_list(bigram(x)+x))
df = df.loc[df.astype(str).drop_duplicates().index]
df['unibi_word'] = df['unibi']
df = df.explode('unibi_word')
df= df[~df["unibi_word"].isin(drop_words)]

In [186]:
counts = df["unibi_word"].value_counts().reset_index(name="counts")

In [200]:
#filter words
filter_words = ["flex",
               "hello",
               ".com",
                "email",
                "sent",
                "wrote",
                "hi",
                "iphone",
                "please",
                "company",
                "thank",
                "you",
                ":",
                ","
               ]

In [201]:
counts = counts[ ~counts["index"].str.contains("|".join(filter_words))]
counts = counts[counts["counts"]>10]

In [203]:
counts_di = dict(zip(counts["index"],counts["counts"]))

In [206]:
df["number of times word appeard"]=df["unibi_word"].map(counts_di)

In [209]:
df = df.dropna()

In [211]:
df.to_csv("flexfit_email_word_frequency.csv")

In [212]:
counts.to_csv("flexfit_word_frequency_only.csv")